<a href="https://colab.research.google.com/github/Karthikreddy1010/kaggle_competitions/blob/main/Binary_Prediction_of_Poisonous_Mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

# **Binary Prediction of Poisonous Mushroom**

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"karthikreddy0101","key":"94c7d7bc87faf1cfa5cf53fe7d0d49e8"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle competitions download -c playground-series-s4e8

100% 82.0M/82.3M [00:02<00:00, 45.0MB/s]
100% 82.3M/82.3M [00:02<00:00, 33.2MB/s]


In [ ]:
!unzip playground-series-s4e8.zip


Archive:  playground-series-s4e8.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df=pd.read_csv('train.csv')
train_df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-root             object 
 13  stem-surface          object 
 14  stem-color            object 
 15  veil-type             object 
 16  veil-color            object 
 17  has-ring              object 
 18  ring-type             object 
 19  spore-print-color     object 
 20  habitat               object 
 21  season                object 
dtypes: float64(3), int64(1), object(18)
memory

In [ ]:
train_df.describe()

,id,cap-diameter,stem-height,stem-width
count,3.116945e+06,3.116941e+06,3.116945e+06,3.116945e+06
mean,1.558472e+06,6.309848e+00,6.348333e+00,1.115379e+01
std,8.997847e+05,4.657931e+00,2.699755e+00,8.095477e+00
min,0.000000e+00,3.000000e-02,0.000000e+00,0.000000e+00
25%,7.792360e+05,3.320000e+00,4.670000e+00,4.970000e+00
50%,1.558472e+06,5.750000e+00,5.880000e+00,9.650000e+00
75%,2.337708e+06,8.240000e+00,7.410000e+00,1.563000e+01
max,3.116944e+06,8.067000e+01,8.872000e+01,1.029000e+02


In [ ]:
from sklearn.impute import SimpleImputer

#to apply for all null values in a dataset
def get_parameters(train_df):
    parameters={}
    for col in train_df.columns[train_df.isnull().any()]:
        if train_df[col].dtype=='float64' or train_df[col].dtype=='int64' or train_df[col].dtype=='int32':
            strategy='mean'
        else:
            strategy='most_frequent'
        missing_values=train_df[col][train_df[col].isnull()].values[0]
        parameters[col]={'missing_values':missing_values,'strategy':strategy}
    return parameters
get_parameters(train_df)

{'cap-diameter': {'missing_values': nan, 'strategy': 'mean'},
 'cap-shape': {'missing_values': nan, 'strategy': 'most_frequent'},
 'cap-surface': {'missing_values': nan, 'strategy': 'most_frequent'},
 'cap-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'does-bruise-or-bleed': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-attachment': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-spacing': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-root': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-surface': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'veil-type': {'missing_values': nan, 'strategy': 'most_frequent'},
 'veil-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'has-ring': {'missing_values': nan, 'strategy': 'most_frequent'},
 'ring-type': {'missing_val

In [ ]:
#to store parameters in new variable
parameters=get_parameters(train_df)

In [ ]:
for col, param in parameters.items():
    missing_values = param['missing_values']
    strategy = param['strategy']
    imp = SimpleImputer(missing_values=missing_values, strategy=strategy)

    transformed_col = imp.fit_transform(train_df[[col]]).reshape(-1)
    train_df[col] = transformed_col


In [ ]:
train_df.isnull().sum()

,0
id,0
class,0
cap-diameter,0
cap-shape,0
cap-surface,0
cap-color,0
does-bruise-or-bleed,0
gill-attachment,0
gill-spacing,0
gill-color,0


In [ ]:

from sklearn.preprocessing import LabelEncoder


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# List of columns to label encode
columns_to_encode = ['class', 'cap-shape', 'cap-surface', 'cap-color',
                     'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing',
                     'gill-color', 'stem-root', 'stem-surface', 'stem-color',
                     'veil-type', 'veil-color', 'has-ring', 'ring-type',
                     'spore-print-color', 'habitat', 'season']

# Apply label encoding to each column
for column in columns_to_encode:
    train_df[column] = label_encoder.fit_transform(train_df[column])




In [ ]:
train_df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,0,8.80,53,72,72,8,44,28,59,...,15,51,55,19,21,5,18,17,25,0
1,1,1,4.51,71,56,64,8,44,28,46,...,15,58,47,19,21,18,39,17,25,3
2,2,0,6.94,53,72,49,8,75,28,59,...,15,51,46,19,21,5,18,17,36,3
3,3,0,3.88,53,81,57,8,70,28,37,...,15,51,55,19,21,5,18,17,25,2
4,4,0,5.85,71,65,74,8,47,28,59,...,15,51,55,19,21,5,18,17,29,0


In [ ]:
X=train_df.drop(columns=['class'])
y=train_df['class']

In [ ]:
from sklearn.preprocessing import MinMaxScaler,RobustScaler

# List of columns to normalize
columns_to_normalize = ['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
                        'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing',
                        'gill-color', 'stem-height', 'stem-width', 'stem-root',
                        'stem-surface', 'stem-color', 'veil-type', 'veil-color',
                        'has-ring', 'ring-type', 'spore-print-color', 'habitat',
                        'season']

# Initialize MinMaxScaler
scaler = RobustScaler()

# Apply the scaler to the columns in X
X[columns_to_normalize] = scaler.fit_transform(X[columns_to_normalize])

# Check the transformed X dataset
print(X.head())


   id  cap-diameter  cap-shape  cap-surface  cap-color  does-bruise-or-bleed  \
0   0      0.619919  -0.777778        -0.20   1.000000                   0.0   
1   1     -0.252033   0.222222        -1.00   0.111111                   0.0   
2   2      0.241870  -0.777778        -0.20  -1.555556                   0.0   
3   3     -0.380081  -0.777778         0.25  -0.666667                   0.0   
4   4      0.020325   0.222222        -0.55   1.222222                   0.0   

   gill-attachment  gill-spacing  gill-color  stem-height  ...  stem-root  \
0        -0.136364           0.0    0.846154    -0.500000  ...        0.0   
1        -0.136364           0.0   -0.153846    -0.397810  ...        0.0   
2         1.272727           0.0    0.846154     0.354015  ...        0.0   
3         1.045455           0.0   -0.846154    -0.627737  ...        0.0   
4         0.000000           0.0    0.846154    -0.916058  ...        0.0   

   stem-surface  stem-color  veil-type  veil-color  has-

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=4)

In [ ]:
import tensorflow as tf
import tensorflow.keras

## Neural Networks

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Predict on the test data
y_pred_probs = model.predict(X_test)

# Convert probabilities to binary class labels (threshold at 0.5)
y_pred_classes = np.where(y_pred_probs > 0.5, 1, 0)

# Generate the classification report
print(classification_report(y_test, y_pred_classes))


19481/19481 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    282310
           1       0.99      0.99      0.99    341079

    accuracy                           0.99    623389
   macro avg       0.99      0.99      0.99    623389
weighted avg       0.99      0.99      0.99    623389



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(21, 1)),  # Adjusted input_shape for Conv1D
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Adjusted output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Check model summary if needed
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 19, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 9, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 7, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 384)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,665 (194.00 KB)

 Trainable params: 49,665 (194.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history1= model.fit(X_train, y_train, epochs=5)


Epoch 1/5
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 167s 2ms/step - accuracy: 0.8581 - loss: 53.8034
Epoch 2/5
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 196s 2ms/step - accuracy: 0.9745 - loss: 0.8886
Epoch 3/5
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 159s 2ms/step - accuracy: 0.9465 - loss: 0.2178
Epoch 4/5
77924/77924 ━━━━━━━━━━━━━━━━━━━━ 159s 2ms/step - accuracy: 0.5474 - loss: 0.6900
Epoch 5/5
75971/77924 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5477 - loss: 0.6886

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Predict on the test data
y_pred_probs = model.predict(X_test)

# Convert probabilities to binary class labels (threshold at 0.5)
y_pred_classes = np.where(y_pred_probs > 0.5, 1, 0)

# Generate the classification report
print(classification_report(y_test, y_pred_classes))


In [ ]:
test_df=pd.read_csv('test.csv')
test_df.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077964 entries, 0 to 2077963
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   cap-diameter          float64
 2   cap-shape             object 
 3   cap-surface           object 
 4   cap-color             object 
 5   does-bruise-or-bleed  object 
 6   gill-attachment       object 
 7   gill-spacing          object 
 8   gill-color            object 
 9   stem-height           float64
 10  stem-width            float64
 11  stem-root             object 
 12  stem-surface          object 
 13  stem-color            object 
 14  veil-type             object 
 15  veil-color            object 
 16  has-ring              object 
 17  ring-type             object 
 18  spore-print-color     object 
 19  habitat               object 
 20  season                object 
dtypes: float64(3), int64(1), object(17)
memory usage: 332.9+ MB


In [ ]:
from sklearn.impute import SimpleImputer

#to apply for all null values in a dataset
def get_parameters(test_df):
    parameters={}
    for col in test_df.columns[test_df.isnull().any()]:
        if test_df[col].dtype=='float64' or test_df[col].dtype=='int64' or train_df[col].dtype=='int32':
            strategy='mean'
        else:
            strategy='most_frequent'
        missing_values=test_df[col][test_df[col].isnull()].values[0]
        parameters[col]={'missing_values':missing_values,'strategy':strategy}
    return parameters
get_parameters(test_df)

{'cap-diameter': {'missing_values': nan, 'strategy': 'mean'},
 'cap-shape': {'missing_values': nan, 'strategy': 'most_frequent'},
 'cap-surface': {'missing_values': nan, 'strategy': 'most_frequent'},
 'cap-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'does-bruise-or-bleed': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-attachment': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-spacing': {'missing_values': nan, 'strategy': 'most_frequent'},
 'gill-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-height': {'missing_values': nan, 'strategy': 'mean'},
 'stem-root': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-surface': {'missing_values': nan, 'strategy': 'most_frequent'},
 'stem-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'veil-type': {'missing_values': nan, 'strategy': 'most_frequent'},
 'veil-color': {'missing_values': nan, 'strategy': 'most_frequent'},
 'has-ring': {'missing_values': n

In [ ]:
#to store parameters in new variable
parameters=get_parameters(test_df)

In [ ]:
for col, param in parameters.items():
    missing_values = param['missing_values']
    strategy = param['strategy']
    imp = SimpleImputer(missing_values=missing_values, strategy=strategy)

    transformed_col = imp.fit_transform(test_df[[col]]).reshape(-1)
    test_df[col] = transformed_col


In [ ]:
test_df.isnull().sum()

,0
id,0
cap-diameter,0
cap-shape,0
cap-surface,0
cap-color,0
does-bruise-or-bleed,0
gill-attachment,0
gill-spacing,0
gill-color,0
stem-height,0


In [ ]:

from sklearn.preprocessing import LabelEncoder


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# List of columns to label encode
columns_to_encode = ['cap-shape', 'cap-surface', 'cap-color',
                     'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing',
                     'gill-color', 'stem-root', 'stem-surface', 'stem-color',
                     'veil-type', 'veil-color', 'has-ring', 'ring-type',
                     'spore-print-color', 'habitat', 'season']

# Apply label encoding to each column
for column in columns_to_encode:
    test_df[column] = label_encoder.fit_transform(test_df[column])




In [ ]:
test_df.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,59,53,44,18,37,17,52,11.13,...,9,45,51,12,21,17,15,18,16,0
1,3116946,6.90,50,53,45,5,37,17,54,1.27,...,9,45,38,12,21,6,14,18,16,0
2,3116947,2.00,36,38,44,5,37,17,41,6.18,...,9,45,38,12,21,6,14,18,16,1
3,3116948,3.47,59,53,44,5,57,17,41,4.98,...,9,45,51,12,14,17,35,18,16,2
4,3116949,6.17,59,39,55,5,55,17,54,6.73,...,9,45,53,12,22,17,14,18,16,2


In [ ]:
# Apply the scaler to the columns in X
test_df[columns_to_normalize] = scaler.fit_transform(test_df[columns_to_normalize])

# Check the transformed X dataset
print(test_df.head())

        id  cap-diameter  cap-shape  cap-surface  cap-color  \
0  3116945      0.589431   0.222222     0.000000      0.000   
1  3116946      0.235772  -0.277778     0.000000      0.125   
2  3116947     -0.760163  -1.055556    -1.071429      0.000   
3  3116948     -0.461382   0.222222     0.000000      0.000   
4  3116949      0.087398   0.222222    -1.000000      1.375   

   does-bruise-or-bleed  gill-attachment  gill-spacing  gill-color  \
0                  13.0        -0.222222           0.0    0.818182   
1                   0.0        -0.222222           0.0    1.000000   
2                   0.0        -0.222222           0.0   -0.181818   
3                   0.0         0.888889           0.0   -0.181818   
4                   0.0         0.777778           0.0    1.000000   

   stem-height  ...  stem-root  stem-surface  stem-color  veil-type  \
0     1.916058  ...        0.0           0.0    0.000000        0.0   
1    -1.682482  ...        0.0           0.0   -1.000000  

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'model' and 'test_df' are already defined and 'model.predict' returns probabilities

# Get predictions from the model
test_predictions = model.predict(test_df)

# Convert predictions to class labels 'p' and 'e'
test_df['class'] = np.where(test_predictions > 0.5, 'p', 'e')

# Save the results to a CSV file
test_df[['id', 'class']].to_csv('submission.csv', index=False)


64937/64937 ━━━━━━━━━━━━━━━━━━━━ 171s 3ms/step
